## Preparar conjunto de datos

El archivo que usaremos para los datos será un excel llamado dataSetEntrenamiento.xls basado en la información contenida en la guía de la Dirección Nacional de Admisiones 'Respuestas estándar 2020.pdf' el cual presenta información relevante para el proceso de admisión de la Universidad Nacional de Colombia para los precesos de pregrado y posgrado, además de la información levantada dentro de la dependencia en formato de pregunta y respuesta.

Para ello vamos a usar el archivo alojado en Google Drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
#Importar librerias
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # The %tensorflow_version magic only works in colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
tf.random.set_seed(1234)

!pip install tensorflow-datasets==1.2.0
import tensorflow_datasets as tfds

import os
import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.9.4
    Uninstalling tensorflow-datasets-4.9.4:
      Successfully uninstalled tensorflow-datasets-4.9.4


In [3]:
import pandas as pd

data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/dataSetEntrenamiento.xlsx')
data

,pregunta,respuesta,respuesta_id,tema_id
0,¿En qué fecha inicia el proceso de admisión?,Podrá consultar esa información en la página w...,0,1
1,¿Cuándo arranca el proceso de admisión?,Podrá consultar esa información en la página w...,0,1
2,¿Desde cuándo se puede iniciar el proceso de a...,Podrá consultar esa información en la página w...,0,1
3,¿En qué momento empieza el proceso de admisión?,Podrá consultar esa información en la página w...,0,1
4,¿A partir de cuándo se puede comenzar el proce...,Podrá consultar esa información en la página w...,0,1
...,...,...,...,...
3992,¿Quién es el encargado de revisar las solicitu...,"Aspirante, con el fin de estudiar su solicitud...",170,23
3993,¿Cuánto tiempo demora el proceso de devolución...,"Aspirante, con el fin de estudiar su solicitud...",170,23
3994,¿Cuáles son las opciones disponibles si no pue...,"Aspirante, con el fin de estudiar su solicitud...",170,23
3995,¿Es posible transferir los derechos de inscrip...,"Aspirante, con el fin de estudiar su solicitud...",170,23


## Cargar y preprocesar datos
Para mantener este ejercicio simple y rápido, estamos limitando el número máximo de muestras de entrenamiento a MAX_SAMPLES=50000 y la longitud máxima de la oración a MAX_LENGTH=40.

Preprocesamos nuestro conjunto de datos en el siguiente orden:

* Se extrae MAX_SAMPLES pares de conversación en una lista de `preguntas` y `respuestas`.
* Se preprocesa cada oración eliminando caracteres especiales en cada oración.
* Se crea tokenizer (asignar texto a ID e ID a texto) usando TensorFlow Datasets SubwordTextEncoder.
* Identifique cada oración y agregue START_TOKEN y END_TOKEN para indicar el comienzo y el final de cada oración.
* Filtre la oración que tenga más de MAX_LENGTH tokens.
* Haga pad a las oraciones tokenizadas a MAX_LENGTH.

In [5]:
import re

# Maximum number of samples to preprocess
MAX_SAMPLES = 10000

def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

def preprocess_sentence(sentence):
  sentence = sentence.lower().strip()
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  sentence = normalize(sentence)
  sentence = re.sub(r"([¿?.!,])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)
  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  sentence = re.sub(r"[^a-zA-Z¿?.!,]+", " ", sentence)
  sentence = sentence.strip()
  # adding a start and an end token to the sentence
  return sentence


def load_questions_answers():
  inputs, outputs = [], []
  for i in data.index:
    inputs.append(preprocess_sentence(data['pregunta'][i]))
    outputs.append(preprocess_sentence(data['respuesta'][i]))
    if len(inputs) >= MAX_SAMPLES:
        return inputs, outputs
  return inputs, outputs

questions, answers = load_questions_answers()

In [6]:
print('Sample question: {}'.format(questions[3800]))
print('Sample answer: {}'.format(answers[3800]))

Sample question: ¿ que documentos debo presentar para solicitar admision a la universidad nacional ?
Sample answer: se invita a ingresar a la pagina web http admisiones . unal . edu . co , pesta a pregrado , en la seccion informacion para aspirantes , vinculo aspirantes que se encuentran fuera de colombia y aspirantes extranjeros que desean estudiar en la universidad . alli se encuentra una guia detallada para diligenciar el formulario de inscripcion .


In [9]:
# Build tokenizer using tfds for both questions and answers
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)

# Define start and end token to indicate the start and end of a sentence
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# Vocabulary size plus start and end token
VOCAB_SIZE = tokenizer.vocab_size + 2


In [10]:
print('Tokenized sample question: {}'.format(tokenizer.encode(answers[9])))


Tokenized sample question: [168, 40, 197, 35, 6, 2, 27, 34, 42, 3, 25, 3, 17, 3, 18, 3, 46, 4, 108, 45, 4, 58, 40, 2, 77, 79, 7, 102]


In [11]:
# Maximum sentence length
MAX_LENGTH = 300


# Tokenize, filter and pad sentences
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # tokenize sentence
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
    # check tokenized sentence max length
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)

  # pad tokenized sentences
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs


questions, answers = tokenize_and_filter(questions, answers)

In [12]:
print('Vocab size: {}'.format(VOCAB_SIZE))
print('Number of samples: {}'.format(len(questions)))

Vocab size: 3919
Number of samples: 3627
